In [ ]:
import datetime
import requests
import pandas as pd
import hopsworks
import great_expectations as ge
import datetime
from pathlib import Path
import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

: 

In [2]:
import dotenv

dotenv.load_dotenv()

AQI_API_KEY = os.getenv("AQI_API_KEY")
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

In [3]:
csv_file = "data/stockholm-st-eriksgatan-83-air-quality.csv"
util.check_file_path(csv_file)

File successfully found at the path: data/stockholm-st-eriksgatan-83-air-quality.csv


In [4]:
country = "sweden"
city = "stockholm"
street = "stockholm-st-eriksgatan-83"
aqicn_url="https://api.waqi.info/feed/@10523"

today = datetime.date.today()
latitude, longitude = util.get_city_coordinates(city)
print("Latitude: ", latitude, "Longitude: ", longitude)

Latitude:  59.33 Longitude:  18.07


In [5]:


project = hopsworks.login()

secrets = util.secrets_api(project.name)
try:
    secrets.create_secret("AQI_API_KEY", AQI_API_KEY)
except hopsworks.RestAPIError:
    AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1157270
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
try:
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
except hopsworks.RestAPIError:
    print("It looks like the AQI_API_KEY doesn't work for your sensor. Is the API key correct? Is the sensor URL correct?")

aq_today_df.head()

,pm25,country,city,street,date,url
0,19.0,sweden,stockholm,stockholm-st-eriksgatan-83,2024-11-12,https://api.waqi.info/feed/@10523


In [7]:
# Save the data to a CSV file
df = pd.read_csv(csv_file, parse_dates=["date"], skipinitialspace=True)

# Merge the data
df = df[["date", "pm25"]]
df["country"] = country
df["city"] = city
df["street"] = street
df["url"] = aqicn_url
df["pm25"] = df["pm25"].astype("float32")

# Let's check the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     2200 non-null   datetime64[ns]
 1   pm25     2183 non-null   float32       
 2   country  2200 non-null   object        
 3   city     2200 non-null   object        
 4   street   2200 non-null   object        
 5   url      2200 non-null   object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 94.7+ KB


In [8]:
# Drop rows with missing values
df.dropna(inplace=True)
df

,date,pm25,country,city,street,url
0,2024-11-01,9.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
1,2024-11-02,9.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
2,2024-11-03,7.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
3,2024-11-04,11.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
4,2024-11-05,16.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
...,...,...,...,...,...,...
2178,2018-06-26,18.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
2179,2018-06-27,23.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
2180,2018-06-28,24.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
2181,2018-06-29,15.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523


In [9]:
earliest_date = pd.Series.min(df['date'])
earliest_date = earliest_date.strftime("%Y-%m-%d")
earliest_date

weather_df = util.get_historical_weather(city, earliest_date, str(today), latitude, longitude)

Coordinates 59.29701232910156°N 18.163265228271484°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [10]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2408 entries, 0 to 2407
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         2408 non-null   datetime64[ns]
 1   temperature_2m_mean          2408 non-null   float32       
 2   precipitation_sum            2408 non-null   float32       
 3   wind_speed_10m_max           2408 non-null   float32       
 4   wind_direction_10m_dominant  2408 non-null   float32       
 5   city                         2408 non-null   object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 94.1+ KB


In [11]:
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite",
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "pm25",
            "min_value": -0.1,
            "max_value": 500.0,
            "strict_min": True,
        },
    )
)

weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

In [12]:
dict_obj = {
    "country": country,
    "city": city,
    "street": street,
    "aqicn_url": aqicn_url,
    "latitude": latitude,
    "longitude": longitude
}

# Convert the dictionary to a JSON string
str_dict = json.dumps(dict_obj)

try:
    secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)
except hopsworks.RestAPIError:
    print("SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("SENSOR_LOCATION_JSON").value
    print(f"{existing_key}")

SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.
{"country": "sweden", "city": "stockholm", "street": "stockholm-st-eriksgatan-83", "aqicn_url": "https://api.waqi.info/feed/@10523", "latitude": 59.33, "longitude": 18.07}


In [13]:
fs = project.get_feature_store()

air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

air_quality_fg.insert(df)

air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("country", "Country where the air quality was measured (sometimes a city in acqcn.org)")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("street", "Street in the city where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city', 'date'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 


# Insert data
weather_fg.insert(weather_df)

weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")


2024-11-12 16:30:40,215 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
2024-11-12 16:30:41,202 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157270/fs/1147973/fg/1345959


Uploading Dataframe: 0.00% |          | Rows 0/2183 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157270/jobs/named/air_quality_1_offline_fg_materialization/executions
2024-11-12 16:30:55,216 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157270/fs/1147973/fg/1345960


Uploading Dataframe: 0.00% |          | Rows 0/2408 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157270/jobs/named/weather_1_offline_fg_materialization/executions
